In [127]:
import requests
from bs4 import BeautifulSoup
import json
import re

Lấy dữ liệu tại trang mytour.vn

Chọn địa điểm là Đà Lạt

Chọn những khách sạn có rating >= 7.5, số người đánh giá >= 10 người

Thời gian checkin, checkout chênh lệnh một ngày

In [128]:
url = 'https://mytour.vn/d352/khach-san-tai-da-lat.html?page={}'
min_rate = 7.5
min_rate_number = 10

Duyệt hết danh sách để lấy sub_url của những khách sạn

Lọc ra những khách sạn có đủ tiêu chuẩn, lưu vào danh sách

In [131]:
f = open('list_hotel.csv', 'w', encoding='utf-8')
f.write('id,name,url,rating,rate_number,star,lat,long\n')

45

In [132]:
count_page = 1
total_record = 0
while True:
    # Duyệt trang mới
    fill_url = url.format(count_page)
    response = requests.get(fill_url)    
    if (count_page > 1) & (response.url != fill_url):
        break
    count_page += 1
    
    # Với mỗi page, lấy danh sách sub_url của các khách sạn
    tree = BeautifulSoup(response.text, 'html.parser')
    data_hotel = tree.find('div', {'id': 'data-hotels', 'class':'hide'})
    list_hotel = data_hotel.find_all('div', {'class': re.compile('product-item*')})

    number_record = 0
    for hotel in list_hotel:
        hotel_id = hotel['data-id']
        hotel_name = hotel.find('h2', {'class': 'title-sm h3'})['title']         
        sub_url = 'https://mytour.vn' + hotel.find('a', {'class':'events-tracking product-name'})['href']
                
        rating = float(0)
        rate_number = 0
        review_tag = hotel.find('div', {'class':'box-review'})
        if review_tag is not None:                   
            rating_tag = review_tag.find('span', {'class':'rate'})        
            if rating_tag is not None:
                rating = float(rating_tag.string)

            rate_number_tag = review_tag.find('p', {'class':'rate-number-info'})
            if rate_number_tag is not None:
                rate_number_tag = rate_number_tag.find('a')
                if rate_number_tag is not None:
                    rate_number_arr = re.findall('(?<!\.)\d+(?!\.)', rate_number_tag.string)
                    if len(rate_number_arr) > 0:
                        rate_number = int(rate_number_arr[0])                
        
        if (rating >= min_rate) & (rate_number >= min_rate_number):
            f.write(hotel_id + ',')  # id
            f.write(hotel_name + ',')
            f.write(sub_url + ',')
            f.write(str(rating) + ',')
            f.write(str(rate_number) + ',')
                        
            star = 0
            star_tag = hotel.find('span', {'class':'star'})
            if star_tag is not None:
                    star_tag = star_tag.find('span')
                    if star_tag is not None:
                        star_arr = re.findall('(?<!\.)\d+(?!\.)', star_tag['class'][0])
                        if len(star_arr) > 0:
                            star = star_arr[0]                      
            f.write(str(star) + ',')           
            
            location_tag = hotel.find('a', {'class':'events-tracking', 'check-place' : str(hotel_id)})
            if location_tag is not None:
                lat = float(location_tag['data-map-lat'])
                lng = float(location_tag['data-map-lng'])                
                f.write(str(lat) + ',')
                f.write(str(lng) + '\n')
            else:
                f.write('-1\t-1\t')
            
            number_record += 1
        
    total_record += number_record
    print('page {} found {} record(s)'.format(count_page - 1, number_record))

print('Finished...Total record(s): {}'.format(total_record))
f.close()

page 1 found 3 record(s)
page 2 found 10 record(s)
page 3 found 11 record(s)
page 4 found 7 record(s)
page 5 found 5 record(s)
page 6 found 9 record(s)
page 7 found 4 record(s)
page 8 found 4 record(s)
page 9 found 1 record(s)
page 10 found 0 record(s)
page 11 found 8 record(s)
page 12 found 5 record(s)
page 13 found 4 record(s)
page 14 found 2 record(s)
page 15 found 5 record(s)
page 16 found 7 record(s)
page 17 found 5 record(s)
page 18 found 8 record(s)
page 19 found 8 record(s)
page 20 found 4 record(s)
page 21 found 7 record(s)
page 22 found 11 record(s)
page 23 found 5 record(s)
page 24 found 9 record(s)
page 25 found 6 record(s)
page 26 found 3 record(s)
page 27 found 0 record(s)
Finished...Total record(s): 151
